# Semantisk Kjerne med OpenBnB MCP Server-integrasjon

Denne notatboken viser hvordan du kan bruke Semantisk Kjerne med den faktiske OpenBnB MCP-serveren for å søke etter ekte Airbnb-overnattingssteder ved hjelp av MCPStdioPlugin. For tilgang til LLM brukes Azure AI Foundry. For å sette opp miljøvariablene dine, kan du følge [Setup Lesson](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Opprette MCP Plugin-tilkoblingen

Vi skal koble til [OpenBnB MCP-serveren](https://github.com/openbnb-org/mcp-server-airbnb) ved hjelp av MCPStdioPlugin. Denne serveren tilbyr søkefunksjonalitet for Airbnb gjennom @openbnb/mcp-server-airbnb-pakken.


## Opprette klienten

I dette eksemplet skal vi bruke Azure AI Foundry for tilgang til LLM. Sørg for at miljøvariablene dine er riktig konfigurert.


## Miljøkonfigurasjon

Konfigurer Azure OpenAI-innstillinger. Sørg for at du har følgende miljøvariabler satt:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Forstå OpenBnB MCP-integrasjonen

Denne notatboken kobler seg til **den ekte OpenBnB MCP-serveren** som tilbyr faktisk Airbnb-søkefunksjonalitet.

### Slik fungerer det:

1. **MCPStdioPlugin**: Bruker standard inn-/utgangskommunikasjon med MCP-serveren
2. **Ekte NPM-pakke**: Laster ned og kjører `@openbnb/mcp-server-airbnb` via npx
3. **Live-data**: Returnerer faktiske Airbnb-eiendomsdata fra deres API-er
4. **Funksjonsoppdagelse**: Agenten oppdager automatisk tilgjengelige funksjoner fra MCP-serveren

### Tilgjengelige funksjoner:

OpenBnB MCP-serveren tilbyr vanligvis:
- **search_listings** - Søk etter Airbnb-eiendommer basert på sted og kriterier
- **get_listing_details** - Hent detaljert informasjon om spesifikke eiendommer
- **check_availability** - Sjekk tilgjengelighet for spesifikke datoer
- **get_reviews** - Hent anmeldelser for eiendommer
- **get_host_info** - Hent informasjon om eiendomsverter

### Forutsetninger:

- **Node.js** installert på systemet ditt
- **Internettforbindelse** for å laste ned MCP-serverpakken
- **NPX** tilgjengelig (følger med Node.js)

### Testing av tilkoblingen:

Du kan teste MCP-serveren manuelt ved å kjøre:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Dette vil laste ned og starte OpenBnB MCP-serveren, som Semantic Kernel deretter kobler seg til for å hente ekte Airbnb-data.


## Kjøre agenten med OpenBnB MCP-serveren

Nå skal vi kjøre AI-agenten som kobler seg til OpenBnB MCP-serveren for å søke etter ekte Airbnb-overnattingssteder i Stockholm for 2 voksne og 1 barn. Du kan gjerne endre `user_inputs`-listen for å tilpasse søkekriteriene.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Sammendrag
Gratulerer! Du har med suksess bygget en AI-agent som integrerer med søk etter overnatting i den virkelige verden ved hjelp av Model Context Protocol (MCP):

## Teknologier brukt:
- Semantic Kernel - For å bygge intelligente agenter med Azure OpenAI
- Azure AI Foundry - For LLM-funksjonalitet og chat-komplettering
- MCP (Model Context Protocol) - For standardisert verktøyintegrasjon
- OpenBnB MCP Server - For ekte Airbnb-søkefunksjonalitet
- Node.js/NPX - For å kjøre den eksterne MCP-serveren

## Hva du har lært:
- MCP-integrasjon: Koble Semantic Kernel-agenter til eksterne MCP-servere
- Sanntids datatilgang: Søke etter faktiske Airbnb-eiendommer via live API-er
- Protokollkommunikasjon: Bruke stdio-kommunikasjon mellom agent og MCP-server
- Funksjonsoppdagelse: Automatisk oppdage tilgjengelige funksjoner fra MCP-servere
- Strømming av svar: Fange opp og logge funksjonskall i sanntid
- HTML-rendering: Formatere agentens svar med stilige tabeller og interaktive visninger

## Neste steg:
- Integrere flere MCP-servere (vær, fly, restauranter)
- Bygge et multi-agent-system som kombinerer MCP- og A2A-protokoller
- Lage tilpassede MCP-servere for dine egne datakilder
- Implementere vedvarende samtaleminne på tvers av økter
- Distribuere agenten til Azure Functions med MCP-serverorkestrering
- Legge til brukergodkjenning og bestillingsfunksjonalitet



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vennligst vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
